# Mistral 8x7B x Autogen

This notebook aims to show how you can run mixtral 8x7B with Autogen.

**Step to run locally:**
- Download a llamafile of the [HuggingFace Repo](https://huggingface.co/award40/mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile/tree/main) - no installations needed, read the model card.  
- `chmod +x mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile`
- `./mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile --port 8081`
<!-- ./llamafile -m mixtral-8x7b-instruct-v0.1.Q3_K_M.gguf -->

Like many of you, i am GPU poor. The goal behind this approach was to have easy access to a good opensourced model with limited GPU resources, like a Macbook Pro M1 32GB.
It's not the full model, but it's the most feasible given the resource constraints - see [here](https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF#provided-files) for notes on performance

In [1]:
import os
import shutil
import autogen 
from autogen import AssistantAgent, UserProxyAgent


work_dir = "./coding"
cache_dir = "./.cache"

if os.path.exists(cache_dir) :
    print('Deleting cache...')
    shutil.rmtree(cache_dir)
if os.path.exists(work_dir) :
    print('Deleting previous work...')
    shutil.rmtree(work_dir)

Deleting cache...


In [2]:
# API Configurations
config_list=[
    {
        "base_url": "http://127.0.0.1:8081/v1",
        "api_key": "NULL",
    }
]

llm_config_mistral={
    "config_list": config_list,
}

llm_config_mistral

{'config_list': [{'base_url': 'http://127.0.0.1:8081/v1', 'api_key': 'NULL'}]}

In [3]:
# Instructions for the two agents
system_messages = {
    "USER_PROXY_SYSTEM_MESSAGE": (
"""
<s>[INST]
Instruction:
Assist in executing code and, upon successful completion, indicate the end of the task.

Guidelines:
- Confirm the successful execution of code before concluding any task.

Communication Etiquette:
- Avoid using phrases like "Thank you" or "You're welcome".
- Do not extend assistance beyond the completed task.
- If appreciation phrases occur, conclude the interaction with "TERMINATE".
- "TERMINATE" must only be used after confirming the successfully executed the code.
</s>[/INST]
"""
    ),
    "CODING_ASSISTANT_SYSTEM_MESSAGE" : (
"""
<s>[INST]
Instruction:
As an AI specialized in programming, generate executable code based on user intent. 

Guidelines:
- Provide complete, executable code without requiring user modifications.
- Clearly indicate the script type (Python) at the start of your code blocks.
- Format code responses like this:
    ```python
    <code>
    ```
- Maintain code in a single block. Avoid multiple sections.
- Exclude predicted outputs; only include code.
- Address code issues through follow-up messages for modifications and corrections.

Communication Etiquette:
- Limit responses to code-related information.
- Omit phrases like "Thank you" or "You're welcome".

Penalties apply for non-compliance with guidelines.
</s>[/INST]
"""
    )
}

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config=llm_config_mistral,
    system_message=system_messages['CODING_ASSISTANT_SYSTEM_MESSAGE'],
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("<|endoftext|>"),
    llm_config=llm_config_mistral,
    code_execution_config={
        "work_dir": work_dir, 
        "use_docker": False
    },
    system_message=system_messages['USER_PROXY_SYSTEM_MESSAGE']
)

### Initiate a chat with a simple python problem

In [7]:
user_message = "Write a fibonacci sequence function in python and run it to print 20 numbers."
coding_runner.initiate_chat(coding_assistant, message=user_message)

coding_runner (to coding_assistant):

Write a fibonacci sequence function in python and run it to print 20 numbers.

--------------------------------------------------------------------------------
coding_assistant (to coding_runner):

```python
def fibonacci_sequence(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence

print(fibonacci_sequence(20))
```


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
coding_runner (to coding_assistant):

exitcode: 0 (execution succeeded)
Code output: 
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


--------------------------------------------------------------------------------
coding_assistant (to coding_runner):

<|endoftext|>

--------------------------------------------------------------------------------
